In [93]:
import pandas as pd
import np
from sklearn.utils import shuffle
from math import floor, ceil
import random
import statistics

In [94]:
input_filename = "/home/mchughj/Downloads/Team Placement - Form Responses 1.csv"
all_players = pd.read_csv(input_filename)

print( f"Read in the player file; players: {len(all_players)}")

mindy = all_players[all_players["Name"] == "Mindy Au"]

Read in the player file; players: 102


In [150]:
class Team:
    def __init__(self, n, g):
        self.n = n
        assert(isinstance(g, pd.core.series.Series))
        self.g = g
        self.tm = []
        self.o = []
        self.d = []
        self.m = []
        
    def _getPlayerName(self, p):
        return p["Name"].strip()
    
    def addPlayer(self, p):
        if isinstance(p, pd.core.frame.DataFrame): 
            assert(len(p) == 1)
            p = p.iloc[0]
        assert(isinstance(p, pd.core.series.Series))
        self.tm.append(p)
        
    def totalSkillValue(self):
        return sum([x['SNV'] for x in self.tm])
    
    def _printNamesInList(self, l):
        ls = [self._getPlayerName(x) for x in l]
        return ", ".join(ls)
    
    def _meetsCriteria(self, o, d, m):
        # As long as the proposed offense, defense, and midfield lists contains everyone
        # then the assignment criteria has been met. 
        return len(o) + len(d) + len(m) == len(self.tm)
        
    def _assignPositionsRec(self, players, o, d, m, max_to_assign_to_position, verbose):
        # Base case first.  All players have been selected.
        if len(players) == 0:
            return (o,d,m)
        
        p = players[0]
        if (len(players) > 1):
            remaining = players[1:]
        else:
            remaining = []
        
        availablePositions = p["Position"].split(",")
        if availablePositions[0] == "No preference":
            availablePositions = ["Offense", "Defense", "Midfield"]
          
        if verbose:
            print( f"assignPositionsRec; p: {p['Name']}, positions: {', '.join(availablePositions)}, o: {self._printNamesInList(o)}, d: {self._printNamesInList(d)}, m: {self._printNamesInList(m)}")
           
        # We only assign a maximum number of people to the same position, except
        # if all positions (or a majority of them - 2) have the maximum number of
        # people assigned to them.  In which case we allow the number assigned to
        # go over.
        #
        # This handles the case where a team has an extra player (because of 
        # just general distribution of players and a team _has to have_ more) 
        # and we want to allow assignment of those additional people to a position.
        num_assigned_to_max = sum(len(x) == max_to_assign_to_position for x in [o,d,m])
        
        for a in availablePositions:
            a = a.strip()
            if verbose:
                print( f"assignPositionsRec; now considering p: {p['Name']}, for a: {a}")
            if (a == "Defense" and (len(d)< max_to_assign_to_position or num_assigned_to_max >= 2)):
                (ro, rd, rm) = self._assignPositionsRec(remaining, o, d + [p], m, max_to_assign_to_position, verbose)
                if self._meetsCriteria(ro, rd, rm):
                    return (ro,rd,rm)
            if (a == "Midfield" and (len(m)< max_to_assign_to_position or num_assigned_to_max >= 2) ):
                (ro, rd, rm) = self._assignPositionsRec(remaining, o, d, m + [p], max_to_assign_to_position, verbose)
                if self._meetsCriteria(ro, rd, rm):
                    return (ro,rd,rm)
            if (a == "Offense" and (len(o)< max_to_assign_to_position or num_assigned_to_max >= 2) ):
                (ro, rd, rm) = self._assignPositionsRec(remaining, o + [p], d, m, max_to_assign_to_position, verbose)
                if self._meetsCriteria(ro, rd, rm):
                    return (ro,rd,rm)
        
                
            if verbose: 
                print( f"_assignPositionsRec; it didn't work out for p: {p['Name']}, for a: {a}")
        if verbose:
            print( f"_assignPositionsRec; didn't work out for p: {p['Name']}, in any positions, o: {len(o)}, d: {len(d)}, m: {len(m)}")
        
        # Falling through to here is an implicit failure of assigning the current person to 
        # a position.  This is fine and handled appropriately by the recursive case.
        return (o,d,m)
            
    def assignPositions(self, max_to_assign_to_position = 3, verbose = False):
        if verbose:
            print (f"assignPositions;  going to call initial; team: {self.n}, names: {self._printNamesInList(self.tm)}")
    
        # Recursive approach with initial list of offense, defense, and midfield 
        # as empty and starting with all players to consider. 
        (o, d, m) = self._assignPositionsRec(self.tm, [], [], [], max_to_assign_to_position, verbose)
        
        if self._meetsCriteria(o,d,m):
            self.o = o
            self.d = d
            self.m = m
        
            return True
        else:
            return False
    
    
    def __str__(self):     
        player_string = ""
        for i,t in enumerate(self.tm):
            name = self._getPlayerName(t)
            player_string += f"  Player {i+1}: {name}\n";

        os = [self._getPlayerName(x) for x in self.o] 
        offense_string = "  Offense: " + ",".join(os)

        ds = [self._getPlayerName(x) for x in self.d] 
        defense_string = "  Defense: " + ",".join(ds)
        
        ms = [self._getPlayerName(x) for x in self.m] 
        midfield_string = "  Midfield: " + ",".join(ms) 
        
        return f"Team #{self.n}\n  Goal: {self._getPlayerName(self.g)}\n{player_string}{offense_string}\n{defense_string}\n{midfield_string}"; 

In [151]:
goal_keepers = all_players[ all_players["Position"] == "Goal keeper" ]
print( f"Found goal keepers; count: {len(goal_keepers)}")

Found goal keepers; count: 12


In [152]:
# Using a random start start state (based on the value passed in for 
# stability) generate one possible solution.
#
# This function returns the number of fully assigned teams along with 
# the list of teams.
def doOne(random_state, verbose = False):
    players = shuffle(all_players, random_state=random_state)

    players.drop(players[players['Position'] == "Goal keeper"].index, inplace = True)
    # Get rid of Mindy since she will be placed in a special way.
    players.drop(players[players['Name'] == "Mindy Au"].index, inplace = True)

    teams = []
    count = 0
    for index, g in goal_keepers.iterrows():
        count += 1
        t = Team(count, g)

        if g["Name"] == "Maha":
            t.addPlayer(mindy)

        teams.append(t)
 
    number_teams = len(goal_keepers)
    max_players_in_team = floor(len(players) / float(number_teams))
    
    if verbose:
        print( f"Going to assign remaining {len(players)} across {len(goal_keepers)} teams.")
        print( f"Max players in team, not counting goalie, is {max_players_in_team}.")
    
    # We don't want to overload any one team by giving it too many members.
    #
    # We initialize the team_count with the current number of teammates in it
    # to handle the case where we have pre-assigned any members.
    team_count = dict(zip(range(1,number_teams+1), [len(x.tm) for x in teams])) 
    team_skill = dict(zip(range(1,number_teams+1), [x.totalSkillValue() for x in teams]))

    def getNextTeam(snv):
        min_members = min(team_count.values())
        min_member_teams = dict(filter(lambda elem: elem[1] == min_members, team_count.items()))
        
        if verbose:
            print( f"getNextTeam; snv: {snv}, min_members: {min_members}, c: {len(min_member_teams)}, min_member_teams: {min_member_teams}")
        
        best_team = list(min_member_teams.keys())[0]
        best_measure = 10000000
        for x in min_member_teams.keys():
            # Consider assigning the current SNV to team 'x'.  
            # What would the variance be then in the skill level?
            skill_levels_under_consideration = [team_skill[k] + ( snv if (k == x) else 0 ) for k in min_member_teams.keys()]
            
            ms = min(skill_levels_under_consideration)
            mx = max(skill_levels_under_consideration)
            
            v = mx - ms
            
            if verbose:
                print( f"   getNextTeam; considering assigning it to {x} results in measure: {v} (due to {skill_levels_under_consideration})")
            
            if v < best_measure:
                best_measure = v
                best_team = x
        
        # Now we know that best_team is the right team to use.
        team_count[best_team] += 1
        team_skill[best_team] += snv
        
        if verbose:
            print( f"getNextTeam; returning team {best_team}.  {list(team_skill.values())}.\n----")
        
        return best_team          

    # Create an array where we assign each skill level to the next available
    # team.  This ensures proper distribution of skills across all of the teams.
    sorted_members = players.sort_values(by=['SNV'], ascending=False) 
    team = [ getNextTeam(sorted_members.loc[x,"SNV"]) for x in sorted_members.index ]

    if verbose:
        # Display some information about the distribution of players to teams.
        h,_ = np.histogram(team, bins=len(goal_keepers))
        print( f"Histogram of team size is: {h})")

    # Add a team column to the players
    sorted_members.insert(1, "Team", value = team)

    # Walk through the team assignments and put them into the instance of the
    # team class.
    for index, p in sorted_members.iterrows():
        teams[p["Team"]-1].addPlayer(p)

    number_works = 0
    for t in teams:
        result = t.assignPositions(max_to_assign_to_position=2, verbose=False)
        if result:
            number_works += 1

    return (number_works, teams)

In [153]:
n, final_teams = doOne(0, False)

team_member_counts = [ len(x.tm) for x in final_teams ]
print (f"Team membership size: {team_member_counts}")

skill_values = [ t.totalSkillValue() for t in final_teams ]
print (f"Team skills: {skill_values}")

print(n)
for t in final_teams:
    print(str(t))


Team membership size: [7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8]
Team skills: [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
12
Team #1
  Goal: Suzie Ness
  Player 1: Eva Gillespie
  Player 2: Tanya Fabian
  Player 3: Shanna Clinton
  Player 4: Dana Shemorry
  Player 5: Alicia Castenada
  Player 6: Taylor Kelleher
  Player 7: Jing Ceci Bissonnette
  Offense: Shanna Clinton,Dana Shemorry
  Defense: Eva Gillespie,Tanya Fabian,Jing Ceci Bissonnette
  Midfield: Alicia Castenada,Taylor Kelleher
Team #2
  Goal: Rachel Emswiler
  Player 1: Jen Bessler
  Player 2: Patty Ryder
  Player 3: Mikala Zinter
  Player 4: Erin Meador
  Player 5: Taryn Essinger
  Player 6: Marissa Bea
  Player 7: Barb Marquardt
  Offense: Jen Bessler,Erin Meador,Marissa Bea
  Defense: Patty Ryder,Mikala Zinter
  Midfield: Taryn Essinger,Barb Marquardt
Team #3
  Goal: Diana Roll
  Player 1: Carolyn Hall
  Player 2: Blaine wolde
  Player 3: Marissa London
  Player 4: Margaret Fraczek
  Player 5: Jessica Bierhaus
  Player 6: M

In [154]:
# If I ever want/need additional solutions then I can do something like the below.
do_larger_search = False
if do_larger_search:
    m = 0
    best_i = 0
    best_teams = None
    for i in range(100):
        (n, teams) = doOne(i)
        if n > m:
            best_i = i
            m = n
            best_teams = teams

        if i % 10 == 0:
            print (f"Done with {i}, best is {best_i} - with {m} good teams.")


    print (f"Best teams is {best_i} and has {m} good teams")
    for t in best_teams:
        print(str(t))
        
    final_teams = best_teams
    

In [155]:

for t in final_teams:
    print(f"\nTeam {t.n}")
    print(f"{t._getPlayerName(t.g)},Goal")
    if t.o:
        for o in t.o:
            print(f"{t._getPlayerName(o)},Offense,{o['SNV']}")
        for o in t.m:
            print(f"{t._getPlayerName(o)},Midfield,{o['SNV']}")
        for o in t.d:
            print(f"{t._getPlayerName(o)},Defense,{o['SNV']}")
    else:
        for o in t.tm:
            print(f"{t._getPlayerName(o)},Unassigned,{o['Position']}")


Team 1
Suzie Ness,Goal
Shanna Clinton,Offense,2
Dana Shemorry,Offense,2
Alicia Castenada,Midfield,2
Taylor Kelleher,Midfield,2
Eva Gillespie,Defense,3
Tanya Fabian,Defense,3
Jing Ceci Bissonnette,Defense,1

Team 2
Rachel Emswiler,Goal
Jen Bessler,Offense,3
Erin Meador,Offense,2
Marissa Bea,Offense,2
Taryn Essinger,Midfield,2
Barb Marquardt,Midfield,1
Patty Ryder,Defense,3
Mikala Zinter,Defense,2

Team 3
Diana Roll,Goal
Carolyn Hall,Offense,3
Blaine wolde,Offense,3
Jessica Bierhaus,Midfield,2
Melissa Hoberg,Midfield,2
Marissa London,Defense,2
Margaret Fraczek,Defense,2
Celina Bednar,Defense,1

Team 4
Christina Toney,Goal
Nancy Le,Offense,3
Jennifer Sonson,Offense,2
Heidi Hasty,Offense,1
Alyssa Meyer,Midfield,2
Crispina M. Foss,Midfield,2
Jamie OBrien,Defense,3
Yibe Yiemunu,Defense,2

Team 5
Heather Johnson,Goal
Kristen Ho,Offense,3
Val spannaus,Offense,2
Amanda Smith,Midfield,2
Clara Lima,Midfield,2
Genevieve Norwood,Defense,3
Lindsay griffin,Defense,2
Maria Simon,Defense,1

Team 6
All

In [88]:
best_teams[8].assignPositions()

True